In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from fake_useragent import UserAgent
from pprint import pprint
import re
import json

In [24]:
ua = UserAgent()
url = 'https://www.gb.ru'
headers = {'User-Agent': ua.random}
params = {'page': 1}
session = requests.Session()
all_posts = []

In [ ]:
ua = UserAgent()
url = 'https://books.toscrape.com/catalogue/'
headers = {'User-Agent': ua.random}
session = requests.Session()
all_books = []
page = "page-1.html"
while True:
    response = session.get(url + page, headers=headers)
    print(f"Обрабатывается  страница {page}", end='\r')
    # soup = BeautifulSoup(response.text, 'html.parser')
    # print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')
    try:
        page = soup.find('li', class_='next').find('a')['href']
    except:
        break
    # print(len(books))
    # books
    for book_id in books:
        page_books = {}
        books_info = book_id.find('a')
        href_result = books_info.get('href')
        # print(href_result)
        response = session.get(url + href_result, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        book = soup.find_all('div', class_='col-sm-6 product_main')

        for book_id2 in book:
            try:
                page_books['title'] = book_id2.find('h1').text
            except:
                page_books['title'] = 'None'
            
            # page_books['stok_info'] = book_id2.find('p', class_='instock availability').find(string=re.compile('In stock ')).get_text(strip=True)
            try:
                in_stock = book_id2.find('p', class_='instock availability').find(string=re.compile('In stock ')).get_text(strip=True)
                match = re.findall(r'\d+', in_stock)
                # Проверяем, найдено ли число, и преобразуем его в int
                if match:
                    page_books['in_stok'] = int(match[0])
                else:
                    page_books['in_stok'] = 0
            except:
                page_books['in_stok'] = 0
            try:
                page_books['price_info'] = book_id2.find('p', class_='price_color').text
            except:
                page_books['price_info'] = 'None'
            try:
                rating_element = book_id2.find('p', {'class': 'star-rating'})
                if rating_element:
                    page_books['rating']=rating_element.get('class')[1]
                else:
                    page_books['rating']='None'
            except:
                    page_books['rating']='None'
            try:
                product_page = soup.find('article', {'class': 'product_page'})                                                       
                paragraph_box =  product_page.find_all('p', {'class': None})
                page_books['description'] = " ".join(paragraph.text.strip() for paragraph in paragraph_box)
            except:
                page_books['description'] = 'None'
            try:
                UPC =  soup.find('table', {'class': 'table table-striped'})
                page_books['UPC'] = UPC.find('td').text
                # print(page_books['UPC'])
            except:
                page_books['UPC'] = 'None'
                # print(page_books['UPC'])

            all_books.append(page_books)
            
# print(all_books)

a897fe39b1053632страница page-1.html
90fa61229261140a
6957f44c3847a760
e00eb4fd7b871a48
4165285e1663650f
f77dbf2323deb740
2597b5a345f45e1b
e72a5dfc7e9267b2
e10e1e165dc8be4a
1dfe412b8ac00530
0312262ecafa5a40
30a7f60cd76ca58c
ce6396b0f23f6ecc
3b1c02bac2a429e6
a34ba96d4081e6a4
deda3e61b9514b83
feb7cc7701ecf901
e30f54cea9b38190
a18a4f574854aced
a22124811bfa8350
23356462d1320d61страница page-2.html
327f68a59745c102
5674a18a29a43ced
c2e46a2ee3b4a322
00bfed9e18bb36f3
9528d0948525bf5f
c7d160c2c0de586f
904208d6aa64b655
4c28def39d850cdf
ccd9ffa25efabdea
6258a1f6a6dcfe50
3c039985229453bf
5dada2b7be26bd03
9f6568e9c95f60b0
6be3beb0793a53e7
668fe56b17cfcd4f
094b269567e1c300
19ed25f4641d5efd
be5cc846f45496fb
55f9da0c5eea2e10
b4fd5943413e089aстраница page-3.html
c8f7f0cb1abb9cac
b12b89017878a60d
c27f6e1f185b0383
acddfeab2a407640
3ecb690d1542c568
8d455c7539795d2a
657fe5ead67a7767
d170b5a0da46380e
7ae099f3898e0209
9a9d6e9bc555731e
09555be379ad9253
7858914fad68493c
51653ef291ab7ddc
709822d0b5bcb7f4
4eed6

KeyboardInterrupt: 

In [12]:
session.close()

In [9]:
# Создание pandas dataframe
df = pd.DataFrame(all_books)
print(df)

                                                 title  in_stok price_info  \
0                                 A Light in the Attic       22    Â£51.77   
1                                   Tipping the Velvet       20    Â£53.74   
2                                           Soumission       20    Â£50.10   
3                                        Sharp Objects       20    Â£47.82   
4                Sapiens: A Brief History of Humankind       20    Â£54.23   
..                                                 ...      ...        ...   
975                             Icing (Aces Hockey #2)        1    Â£40.44   
976  Hawkeye, Vol. 1: My Life as a Weapon (Hawkeye #1)        1    Â£45.24   
977  Having the Barbarian's Baby (Ice Planet Barbar...        1    Â£34.96   
978               Giant Days, Vol. 1 (Giant Days #1-4)        1    Â£56.76   
979           Fruits Basket, Vol. 1 (Fruits Basket #1)        1    Â£40.28   

    rating                                        description  

In [14]:
df.to_csv('books.csv', sep='|')
with open('books.json', 'w', encoding='utf-8') as f:
    json.dump(all_books, f, indent=4)

In [179]:
ua = UserAgent()
url = 'https://books.toscrape.com/catalogue/'
headers = {'User-Agent': ua.random}
session = requests.Session()
all_books = []
response = session.get(url + 'page-1.html', headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
books = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')
print(len(books))
# books
for book_id in books:
    page_books = {}
    books_info = book_id.find('a')
    href_result = books_info.get('href')
    # print(href_result)
    response = session.get(url + href_result, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    book = soup.find_all('div', class_='col-sm-6 product_main')

    for book_id2 in book:
        try:
            page_books['title'] = book_id2.find('h1').text
        except:
            page_books['title'] = 'None'
        
        # page_books['stok_info'] = book_id2.find('p', class_='instock availability').find(string=re.compile('In stock ')).get_text(strip=True)
        try:
            in_stock = book_id2.find('p', class_='instock availability').find(string=re.compile('In stock ')).get_text(strip=True)
            match = re.findall(r'\d+', in_stock)
            # Проверяем, найдено ли число, и преобразуем его в int
            if match:
                page_books['in_stok'] = int(match[0])
            else:
                page_books['in_stok'] = 0
        except:
            page_books['in_stok'] = 0
        try:
            page_books['price_info'] = book_id2.find('p', class_='price_color').text
        except:
            page_books['price_info'] = 'None'
        try:
            rating_element = book_id2.find('p', {'class': 'star-rating'})
            if rating_element:
                page_books['rating']=rating_element.get('class')[1]
            else:
                page_books['rating']='None'
        except:
                page_books['rating']='None'
        try:
            product_page = soup.find('article', {'class': 'product_page'})                                                       
            paragraph_box =  product_page.find_all('p', {'class': None})
            page_books['description'] = " ".join(paragraph.text.strip() for paragraph in paragraph_box)
        except:
            page_books['description'] = 'None'

        all_books.append(page_books)
print(all_books)

200
20
[{'title': 'A Light in the Attic', 'in_stok': 22, 'price_info': 'Â£51.77', 'rating': 'Three', 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And 

In [4]:
while True:
    response = session.get(url+"/posts", headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    posts = soup.find_all('div', class_='post-item')
    if not posts:
        break
    # else:
    #     params['page'] +=1
    for post in posts:
        post_info = {}
        try: 
            name_info = post.find('a', class_='post-item__title')
            post_info['name'] = name_info.getText()
        except:
            post_info['name'] = 'None'
        try:
            post_info['url'] = url + name_info.get('href')
        except:
            post_info['url'] = 'None'
        try:
            add_info = post.find('div',class_='text-muted').findChildren('span')
            post_info['views'] = int(add_info[0].getText())
        except:
            post_info['views'] = -1
        try:
            post_info['comments'] = int(add_info[1].getText())
        except:
            post_info['comments'] = -1
        all_posts.append(post_info)
    print(f"Обработана {params['page']} страница", end='\r')
    params['page'] +=1

In [ ]:
session.close()

In [ ]:
pprint(all_posts)
pprint(len(all_posts))

In [5]:
# Создание pandas dataframe
df = pd.DataFrame(all_posts)
print(df)


                                                   name  \
0           Как в 13 лет заработать на 3D-моделировании   
1             Как совмещать работу, семью и обучение ИТ   
2     Закончить GeekSchool, чтобы помогать маме с би...   
3     Маркетолог Gucci, курьер, аналитик ― карьера н...   
4     Как выступления помогают профессиональному раз...   
...                                                 ...   
2058       Личные качества vs профессиональные качества   
2059    Лучшие инструменты по достижению целей. Часть 1   
2060                     7 элементов успешного обучения   
2061          7 типичных ошибок неопытных программистов   
2062                   Цели, которым не суждено сбыться   

                                                    url  views  comments  
0             https://www.gb.ru/posts/how-make-3d-money  51691        -1  
1                    https://www.gb.ru/posts/howmakeall  52629         6  
2                 https://www.gb.ru/posts/buisnesformom  53150    

In [7]:
df.head()

,name,url,views,comments
0,Как в 13 лет заработать на 3D-моделировании,https://www.gb.ru/posts/how-make-3d-money,51691,-1
1,"Как совмещать работу, семью и обучение ИТ",https://www.gb.ru/posts/howmakeall,52629,6
2,"Закончить GeekSchool, чтобы помогать маме с би...",https://www.gb.ru/posts/buisnesformom,53150,5
3,"Маркетолог Gucci, курьер, аналитик ― карьера н...",https://www.gb.ru/posts/guccikurieranalytics,51693,3
4,Как выступления помогают профессиональному раз...,https://www.gb.ru/posts/publictalk,53964,1


In [8]:
df.to_csv('gb_posts.csv')

In [21]:
df[df['views'] == df['views'].max()]


,name,url,views,comments
1743,Как правильно настроить монитор,https://www.gb.ru/posts/how_to_setup_monitor,1083816,19


In [22]:
df[df['views'] == df['views'].min()]

,name,url,views,comments
1453,Ученики в восторге!,https://www.gb.ru/posts/english_programmer,29,0
